https://nanonets.com/blog/human-pose-estimation-2d-guide/

Human Pose estimation is an important problem since it is a crucial step towards understanding people in images and videos. 


#### What is Human Pose Estimation?

Human Pose Estimation is defined as the problem of localization of human joints (also known as keypoints - elbows, wrists, etc) in images or videos. It is also defined as the search for a specific pose in space of all articulated poses.

* 2D Pose Estimation - Estimate a 2D pose (x,y) coordinates for each joint from a RGB image.


* 3D Pose Estimation - Estimate a 3D pose (x,y,z) coordinates a RGB image.

#### Why is it hard?

Strong articulations, small and barely visible joints, occlusions, clothing, and lighting changes make this a difficult problem.

#### Different approaches to 2D Human Pose Estimation

#### Classical approaches
The classical approach to articulated pose estimation is using the pictorial structures framework. The basic idea here is to represent an object by a collection of "parts" arranged in a deformable configuration (not rigid). A "part" is an appearance template which is matched in an image. When parts are parameterized by pixel location and ***orientation***, the resulting structure can model articulation which is very relevant in pose estimation. (A structured prediction task)

> pixel orientation???

The above method, however, comes with the ***limitation of having a pose model not depending on image data***. As a result, research has focused on enriching the representational power of the models.

> pose model???

***Deformable part models - [Yang and Ramanan](https://www.cs.cmu.edu/~deva/papers/pose_pami.pdf) use a mixture model of parts which expresses complex joint relationships. Deformable part models are a collection of templates arranged in a deformable configuration and each model has global template + part templates. These templates are matched for in an image to recognize/detect an object. The Part-based model can model articulations well. This is however achieved at the cost of limited expressiveness and does not take in global context into account.***

> what???

#### Deep Learning based approaches

The classical pipeline has its limitations and Pose estimation has been greatly reshaped by CNNs. With the introduction of [“DeepPose” by Toshev et al](https://arxiv.org/pdf/1312.4659.pdf), research on human pose estimation began to shift from classic approaches to Deep Learning. Most of the recent pose estimation systems have universally adopted ConvNets as their main building block, largely replacing hand-crafted features and graphical models; this strategy has yielded drastic improvements on standard benchmarks.

#### papers must be read

1. [DeepPose](https://arxiv.org/pdf/1312.4659.pdf)

2. [Efficient Object Localization Using Convolutional Networks](https://arxiv.org/pdf/1411.4280.pdf)

3. [Convolutional Pose Machines](https://arxiv.org/pdf/1602.00134.pdf)

4.    Human Pose Estimation with Iterative Error Feedback

5.    Stacked Hourglass Networks for Human Pose Estimation

6.   Simple Baselines for Human Pose Estimation and Tracking

7.    Deep High-Resolution Representation Learning for Human Pose Estimation

#### DeepPose: Human Pose Estimation via Deep Neural Networks (CVPR’14) 

DeepPose was the first major paper that applied Deep Learning to Human pose estimation. It achieved SOTA performance and beat existing models. In this approach, pose estimation is formulated as a CNN-based regression problem towards body joints. They also use a cascade of such regressors to refine the pose estimates and get better estimates. One important thing this approach does is the reason about pose in a holistic fashion, i.e even if certain joints are hidden, they can be estimated if the pose is reasoned about holistically. The paper argues that CNNs naturally provide this sort of reasoning and demonstrate strong results.

#### Model
The model consisted of an AlexNet backend (7 layers) with an extra final layer that outputs 2k joint coordinates - 
$(x_i, y_i)∗2$ for $i \in \{1, 2, ... k\}$ (where $k$ is the number of joints).

The model is trained using a $L2$ loss for regression.

<img src='./imgs/deeppose1.png'>

An interesting idea this model implements is refinement of the predictions using cascaded regressors. Initial coarse pose is refined and a better estimate is achieved. Images are cropped around the predicted joint and fed to the next stage, in this way the subsequent pose regressors see higher resolution images and thus learn features for finer scales which ultimately leads to higher precision.

<img src='./imgs/deeppose2.png'>

#### Results

PCP is used on LSP (Leeds sports dataset) and FLIC (Frames Labeled In Cinema). Have a look at the appendix to find the definitions of some of the popular evaluation metrics like PCP & PCK.

> send refined values to next stage???

> Are joint coordinates already known? why is input image not cropped by those coordinates in the first time?

#### Comments

* This paper applied Deep Learning (CNN) to Human Pose Estimation and pretty much kicked off research in this direction. 


* Regressing to $X, Y$ locations is difficult and adds learning complexity which weakens generalization and hence performs poorly in certain regions. 


* Recent SOTA methods transform the problem to estimating $K$ heatmaps of size $W_0 \times H_0, \{H_1,H_2, ..., H_k\}$, where each heatmap $H_k$ indicates the location confidence of the $k$th keypoint$. ($K$ keypoints in total).

> heatmap??

#### Efficient Object Localization Using Convolutional Networks (CVPR’15)

This approach generates heatmaps by running an image through ***multiple resolution banks*** in parallel to simultaneously capture features at a variety of scales. ***The output is a discrete heatmap instead of continuous regression.*** A heatmap predicts the probability of the joint occurring at each pixel. This output model is very successful and a lot of the papers that followed predict heatmaps instead of direct regression.

> multiple resolution banks?

> predict heatmap???

<img src='./imgs/pose4.png'>


#### Model

A multi-resolution CNN architecture (coarse heatmap model) is used to implement a ***sliding window detector to produce a coarse heatmap output***.

The main motivation of this paper is to recover the ***spatial accuracy lost due to pooling*** in the initial model. They do this by using an additional ‘pose refinement’ ConvNet that refines the localization result of the coarse heat-map. However, unlike a standard cascade of models, they ***reuse existing convolution features***. This not only reduces the number of trainable parameters in the cascade but also acts as a regulariser for the coarse heat-map model since the coarse and fine models are trained jointly.

In essence, the model consists of the heat-map-based parts model for coarse localization, a module to sample and crop the convolution features at a specified $(x, y)$ location for each joint, as well as an additional convolutional model for fine-tuning.

<img src='./imgs/pose3.png'>

A critical feature of this method is the joint use of a ConvNet and a ***graphical model***. The graphical model learns typical spatial relationships between joints.

> graphical model???

#### Training

The model is trained by minimizing the `Mean Squared-Error (MSE)` distance of our predicted heat-map to a target heat-map (The target is a 2D Gaussian of constant variance (σ ≈ 1.5 pixels) centered at the ground-truth $(x, y)$ joint location)

#### Comments

* Heatmaps work better than direct joint regression


* Joint use of a CNN and Graphical Model


* However, these methods lack ***structure modelling***. The space of 2D human poses is highly structured because of body part proportions, left-right symmetries, interpenetration constraints, joint limits (e.g. elbows do not bend back) and physical connectivity (e.g. wrists are rigidly related to elbows), among others. Modelling this structure should make it easier to pinpoint the visible keypoints and make it possible to estimate the occluded ones. The next few papers tackle this, in their own novel ways.

#### Convolutional Pose Machines (CVPR’16)

This is an interesting paper that uses something called a Pose machine. A pose machine consists of an image feature computation module followed by a prediction module. Convolutional Pose Machines are completely differentiable and their multi-stage architecture can be trained end to end. They provide a sequential prediction framework for learning rich implicit spatial models and work very well for Human pose. One of the main motivations of this paper is to learn ***long range spatial relationships and they show that this can be achieved by using larger receptive fields.***

> long range spatial relationships?

> larger receptive fields?



#### Model

<img src='./imgs/pose5.png'>

***$g1()$ and $g2()$ predict heatmaps (belief maps in the paper). Above is a high level view. Stage 1 is the image feature computation module, and Stage 2 is the prediction module. Below is a detailed architecture. Notice how the receptive fields increase in size?***

<img src='./imgs/pose6.png'>

***A CPM can consist of > 2 Stages, and the number of stages is a hyperparameter. (Usually = 3). Stage 1 is fixed and stages > 2 are just repetitions of Stage 2. Stage 2 take heatmaps and image evidence as input. The input heatmaps add spatial context for the next stage. (Has been discussed in detail in the paper).***

***On a high level, the CPM refines the heatmaps through subsequent stages.***

***The paper used intermediate supervision after each stage to avoid the problem of vanishing gradients, which is a common problem for deep multi-stage networks.***

<img src='./imgs/pose7.png'>


#### Results

* MPII:  PCKh-0.5 score achieves state of the art at 87.95%, which is 6.11% higher than the closest competitor, and it is noteworthy that on the ankle (the most challenging part), our PCKh@0.5 score is 78.28% which is 10.76% higher than the closest competitor.


* LSP: Model achieves state of the art at 84.32% (90.5% when adding MPII training data).

#### Comments

* Introduced a novel CPM framework that showed SOTA performance of MPII, FLIC and LSP datasets.